In [5]:
import os
import glob
from PIL import Image
import numpy as np
import cv2

# =============================================================================
# --- CONFIGURATION ---
# =============================================================================

# 1. Define the input folders for the red and green channel images.
RED_CHANNEL_FOLDER = "/Users/jyzerresico/chenlab/Synthetic Division/FigS5e/FigS5d_raw/c1"
GREEN_CHANNEL_FOLDER = "/Users/jyzerresico/chenlab/Synthetic Division/FigS5e/FigS5d_raw/c2"

# 2. Define the full paths for the output multi-frame TIFF files.
OUTPUT_RED_TIFF_PATH = "/Users/jyzerresico/chenlab/Synthetic Division/FigS5e/FigS5d_raw/crop_images/red_crop.tif"
OUTPUT_GREEN_TIFF_PATH = "/Users/jyzerresico/chenlab/Synthetic Division/FigS5e/FigS5d_raw/crop_images/green_crop.tif"

# 3. Set the key processing parameters.
#    This threshold is used to identify the bright regions in the green channel.
#    Adjust this value (0-255) based on your images' brightness.
INTENSITY_THRESHOLD = 1

#    Define the size of the final cropped images (Width, Height).
CROP_SIZE = (26, 10)

# =============================================================================
# Core logic for image processing.
# =============================================================================

def find_centroid(frame_array, threshold_value):
    """
    Finds the centroid of the brightest region in an image based on a threshold.

    Args:
        frame_array (np.array): The input image as a NumPy array.
        threshold_value (int): The pixel intensity value (0-255) to use for thresholding.

    Returns:
        tuple: The (x, y) coordinates of the centroid, or None if no contour is found.
    """
    # Apply a binary threshold to the image.
    _, mask = cv2.threshold(frame_array, threshold_value, 255, cv2.THRESH_BINARY)
    
    # Find all contours in the resulting mask.
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Find the largest contour by area.
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Calculate the moments of the largest contour.
        moments = cv2.moments(largest_contour)
        
        # Calculate the centroid coordinates.
        if moments["m00"] != 0:
            cx = int(moments["m10"] / moments["m00"])
            cy = int(moments["m01"] / moments["m00"])
            return (cx, cy)
            
    return None

def calculate_crop_region(centroid, crop_dims, image_dims):
    """
    Calculates the bounding box for cropping around a centroid.

    Args:
        centroid (tuple): The (x, y) coordinates of the center point.
        crop_dims (tuple): The desired (width, height) of the crop.
        image_dims (tuple): The (height, width) of the original image.

    Returns:
        tuple: The (x1, y1, x2, y2) coordinates for the crop region.
    """
    x, y = centroid
    img_height, img_width = image_dims
    crop_width, crop_height = crop_dims
    
    half_width = crop_width // 2
    half_height = crop_height // 2
    
    # Calculate coordinates, ensuring they stay within the image boundaries.
    x1 = max(0, x - half_width)
    y1 = max(0, y - half_height)
    x2 = min(img_width, x + half_width)
    y2 = min(img_height, y + half_height)
    
    return (x1, y1, x2, y2)

def crop_frame(frame_array, crop_region):
    """
    Crops a NumPy array using the given region coordinates.
    """
    x1, y1, x2, y2 = crop_region
    return frame_array[y1:y2, x1:x2]

def save_frames_to_tiff(frames, output_path):
    """
    Saves a list of PIL Image objects to a multi-page TIFF file.
    """
    if not frames:
        print(f"Warning: No frames were provided to save to {os.path.basename(output_path)}.")
        return

    # Ensure the output directory exists.
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    # Save the first frame, and append the rest.
    frames[0].save(
        output_path,
        save_all=True,
        append_images=frames[1:],
        compression="tiff_deflate",
        loop=0
    )
    print(f"Successfully saved {len(frames)} frames to: {os.path.basename(output_path)}")

# =============================================================================
# --- MAIN EXECUTION ---
# =============================================================================

def main():
    """
    Main function to run the image processing workflow.
    """
    print("--- Starting Image Processing Workflow ---")
    
    # 1. Find and sort the input image files.
    print("Locating and sorting image files...")
    red_filepaths = sorted(glob.glob(os.path.join(RED_CHANNEL_FOLDER, '*.tif*')))
    green_filepaths = sorted(glob.glob(os.path.join(GREEN_CHANNEL_FOLDER, '*.tif*')))

    # 2. Validate the file lists.
    if len(red_filepaths) != len(green_filepaths) or not red_filepaths:
        print("\nError: File validation failed.")
        if not red_filepaths:
            print(f"-> No images found in the specified folders.")
        else:
            print(f"-> Mismatch in file counts: {len(red_filepaths)} red vs. {len(green_filepaths)} green images.")
        print("Please check the CONFIGURATION paths at the top of the script.")
        return # Exit the function

    print(f"Found {len(red_filepaths)} pairs of images. Configuration looks good.")

    # 3. Process each pair of frames.
    processed_red_frames = []
    processed_green_frames = []
    
    print("\nProcessing frames...")
    for i, (green_path, red_path) in enumerate(zip(green_filepaths, red_filepaths)):
        # Open images in grayscale mode ('L').
        green_img = Image.open(green_path).convert('L')
        red_img = Image.open(red_path).convert('L')
        
        green_array = np.array(green_img)
        red_array = np.array(red_img)
        
        # Find the centroid in the green channel image.
        centroid = find_centroid(green_array, INTENSITY_THRESHOLD)
        
        if centroid:
            print(f"  Frame {i+1:03d}: Centroid found at {centroid}. Cropping...")
            
            # Calculate the crop region based on the centroid.
            crop_region = calculate_crop_region(centroid, CROP_SIZE, green_array.shape)
            
            # Crop both the red and green frames using the same region.
            cropped_red_array = crop_frame(red_array, crop_region)
            cropped_green_array = crop_frame(green_array, crop_region)
            
            # Convert arrays back to PIL Images and append to the lists.
            processed_red_frames.append(Image.fromarray(cropped_red_array))
            processed_green_frames.append(Image.fromarray(cropped_green_array))
        else:
            print(f"  Frame {i+1:03d}: No centroid found. Skipping frame.")

    # 4. Save the processed frames to new TIFF files.
    print("\nSaving processed image stacks...")
    save_frames_to_tiff(processed_green_frames, OUTPUT_GREEN_TIFF_PATH)
    save_frames_to_tiff(processed_red_frames, OUTPUT_RED_TIFF_PATH)
    
    print("\n--- Workflow finished successfully! ---")


if __name__ == "__main__":
    main()

--- Starting Image Processing Workflow ---
Locating and sorting image files...
Found 204 pairs of images. Configuration looks good.

Processing frames...
  Frame 001: Centroid found at (48, 50). Cropping...
  Frame 002: Centroid found at (48, 49). Cropping...
  Frame 003: Centroid found at (49, 49). Cropping...
  Frame 004: Centroid found at (49, 49). Cropping...
  Frame 005: Centroid found at (49, 49). Cropping...
  Frame 006: Centroid found at (49, 49). Cropping...
  Frame 007: Centroid found at (48, 49). Cropping...
  Frame 008: Centroid found at (48, 49). Cropping...
  Frame 009: Centroid found at (49, 49). Cropping...
  Frame 010: Centroid found at (48, 49). Cropping...
  Frame 011: Centroid found at (49, 49). Cropping...
  Frame 012: Centroid found at (48, 49). Cropping...
  Frame 013: Centroid found at (48, 49). Cropping...
  Frame 014: Centroid found at (49, 49). Cropping...
  Frame 015: Centroid found at (48, 49). Cropping...
  Frame 016: Centroid found at (49, 49). Cropping..